1. Start Spark

In [11]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("RetailDemandIngestion") \
    .getOrCreate()


Load Raw Data (Excel → Spark)

In [12]:
import pandas as pd

pdf = pd.read_excel("../data/raw/Online_Retail.xlsx")
df = spark.createDataFrame(pdf)

df.printSchema()
df.show(5)


root
 |-- InvoiceNo: string (nullable = true)
 |-- StockCode: string (nullable = true)
 |-- Description: string (nullable = true)
 |-- Quantity: long (nullable = true)
 |-- InvoiceDate: timestamp (nullable = true)
 |-- UnitPrice: double (nullable = true)
 |-- CustomerID: double (nullable = true)
 |-- Country: string (nullable = true)

+---------+---------+--------------------+--------+-------------------+---------+----------+--------------+
|InvoiceNo|StockCode|         Description|Quantity|        InvoiceDate|UnitPrice|CustomerID|       Country|
+---------+---------+--------------------+--------+-------------------+---------+----------+--------------+
|   536365|   85123A|WHITE HANGING HEA...|       6|2010-12-01 08:26:00|     2.55|   17850.0|United Kingdom|
|   536365|    71053| WHITE METAL LANTERN|       6|2010-12-01 08:26:00|     3.39|   17850.0|United Kingdom|
|   536365|   84406B|CREAM CUPID HEART...|       8|2010-12-01 08:26:00|     2.75|   17850.0|United Kingdom|
|   536365|   8

26/01/19 10:09:49 WARN TaskSetManager: Stage 54 contains a task of very large size (3431 KiB). The maximum recommended task size is 1000 KiB.


Standardize Column Names

In [13]:
from pyspark.sql.functions import col

df_clean = df.select([
    col(c).alias(c.lower().replace(" ", "_"))
    for c in df.columns
])

df_clean.printSchema()


root
 |-- invoiceno: string (nullable = true)
 |-- stockcode: string (nullable = true)
 |-- description: string (nullable = true)
 |-- quantity: long (nullable = true)
 |-- invoicedate: timestamp (nullable = true)
 |-- unitprice: double (nullable = true)
 |-- customerid: double (nullable = true)
 |-- country: string (nullable = true)



Remove Invalid Rows

In [14]:
df_valid = df_clean.filter(
    (col("quantity") > 0) &
    (col("unitprice") > 0) &
    col("invoiceno").isNotNull() &
    col("invoicedate").isNotNull()
)

df_valid.count()


26/01/19 10:11:09 WARN TaskSetManager: Stage 55 contains a task of very large size (3431 KiB). The maximum recommended task size is 1000 KiB.


530104

Handle Missing Customer IDs

In [15]:
from pyspark.sql.functions import when

df_valid = df_valid.withColumn(
    "customerid",
    when(col("customerid").isNull(), -1).otherwise(col("customerid"))
)


Create Revenue Column

In [16]:
from pyspark.sql.functions import round

df_enriched = df_valid.withColumn(
    "revenue",
    round(col("quantity") * col("unitprice"), 2)
)

df_enriched.select("quantity", "unitprice", "revenue").show(5)


26/01/19 10:12:15 WARN TaskSetManager: Stage 58 contains a task of very large size (3431 KiB). The maximum recommended task size is 1000 KiB.


+--------+---------+-------+
|quantity|unitprice|revenue|
+--------+---------+-------+
|       6|     2.55|   15.3|
|       6|     3.39|  20.34|
|       8|     2.75|   22.0|
|       6|     3.39|  20.34|
|       6|     3.39|  20.34|
+--------+---------+-------+
only showing top 5 rows



26/01/19 10:12:19 WARN PythonRunner: Detected deadlock while completing task 0.0 in stage 58 (TID 113): Attempting to kill Python Worker


Save Clean Data

In [17]:
df_enriched.write \
    .mode("overwrite") \
    .parquet("../data/processed/retail_sales_clean")


26/01/19 10:12:55 WARN TaskSetManager: Stage 59 contains a task of very large size (3431 KiB). The maximum recommended task size is 1000 KiB.
26/01/19 10:12:55 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers
26/01/19 10:12:55 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 84.44% for 9 writers
26/01/19 10:12:55 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 76.00% for 10 writers
26/01/19 10:12:55 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 84.44% for 9 writers
26/01/19 10:12:55 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers
